# 作業 4

讀取資料。

In [1]:
import pandas as pd
import numpy as np
import matplotlib_inline

pd.options.mode.copy_on_write = True
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [2]:
cust_data = pd.read_csv('cust-data.csv', dtype={
    "CustomerID": "string",
    "SaleNum": "string",
})

cust_data

,CustomerID,SaleDate,SaleNum,Code,Item Description,Quantity,UnitPrice,Country
0,12346,1/18/2011 10:01,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,1.04,United Kingdom
1,12347,10/31/2011 12:25,573511,84992,72 SWEETHEART FAIRY CAKE CASES,24,0.55,Iceland
2,12352,9/20/2011 14:34,567505,21314,SMALL GLASS HEART TRINKET POT,8,2.10,Norway
3,12354,4/21/2011 13:11,550911,84997D,CHILDRENS CUTLERY POLKADOT PINK,4,4.15,Spain
4,12354,4/21/2011 13:11,550911,22668,PINK BABY BUNTING,5,2.95,Spain
...,...,...,...,...,...,...,...,...
7520,<NA>,10/3/2011 16:14,569374,23604,SET 10 MINI SANTA & SNOWMAN 17087,3,1.66,United Kingdom
7521,<NA>,1/20/2011 18:01,541695,85059,FRENCH ENAMEL WATER BASIN,3,3.29,United Kingdom
7522,<NA>,7/29/2011 16:00,561820,21109,LARGE CAKE TOWEL CHOCOLATE SPOTS,1,13.29,United Kingdom
7523,<NA>,9/28/2011 16:19,568719,20996,JAZZ HEARTS ADDRESS BOOK,1,0.83,United Kingdom


目的是看看每個客戶都會買什麼，所以我們先根據 CustomerID 和 Code 進行分組。

In [3]:
# 1 CustomerID -> [Multiple Code]

cust_data_grouped = cust_data.groupby('CustomerID')[' Item Description'].apply(list).to_list()

cust_data_grouped[:5]

[['MEDIUM CERAMIC TOP STORAGE JAR'],
 ['72 SWEETHEART FAIRY CAKE CASES'],
 ['SMALL GLASS HEART TRINKET POT'],
 ['CHILDRENS CUTLERY POLKADOT PINK',
  'PINK BABY BUNTING',
  'CAKE STAND WHITE TWO TIER LACE'],
 ['ALARM CLOCK BAKELIKE PINK', "BLACK SIL'T SQU CANDLE PLATE "]]

將產品轉換成 one-hot encoding。

In [4]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()

te_ary = te.fit(cust_data_grouped).transform(cust_data_grouped)
assert isinstance(te_ary, np.ndarray)

df_trans = pd.DataFrame(te_ary, columns=te.columns_)
df_trans


,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,...,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART FLOWER T-LIGHT HOLDER,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2134,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2135,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2136,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


使用 FP-Growth 演算法找出所有最大高頻項目集，進而推導規則

In [5]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(df_trans, min_support=0.0006, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.002806,(MEDIUM CERAMIC TOP STORAGE JAR)
1,0.004210,(72 SWEETHEART FAIRY CAKE CASES)
2,0.002339,(SMALL GLASS HEART TRINKET POT)
3,0.003274,(CHILDRENS CUTLERY POLKADOT PINK)
4,0.000935,(PINK BABY BUNTING)
...,...,...
1809,0.000935,"(WATERING CAN BLUE ELEPHANT, RETROSPOT TEA SET..."
1810,0.000935,"(DELUXE SEWING KIT , RETROSPOT TEA SET CERAMIC..."
1811,0.000935,"(GARLAND WOODEN HAPPY EASTER, LOVE BUILDING BL..."
1812,0.000935,"(POLYESTER FILLER PAD 45x30cm, RETROSPOT RED W..."


執行關聯規則。

In [6]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.23)

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(FAMILY ALBUM WHITE PICTURE FRAME, 72 SWEETHEA...",(CHARLOTTE BAG SUKI DESIGN),0.000935,0.004210,0.000935,1.00,237.555556,0.000932,inf,0.996723
1,"(FAMILY ALBUM WHITE PICTURE FRAME, CHARLOTTE B...",(72 SWEETHEART FAIRY CAKE CASES),0.000935,0.004210,0.000935,1.00,237.555556,0.000932,inf,0.996723
2,"(72 SWEETHEART FAIRY CAKE CASES, CHARLOTTE BAG...",(FAMILY ALBUM WHITE PICTURE FRAME),0.000935,0.004210,0.000935,1.00,237.555556,0.000932,inf,0.996723
3,"(RETROSPOT TEA SET CERAMIC 11 PC , 72 SWEETHEA...",(NATURAL SLATE HEART CHALKBOARD ),0.000935,0.006080,0.000935,1.00,164.461538,0.000930,inf,0.994850
4,"(72 SWEETHEART FAIRY CAKE CASES, NATURAL SLATE...",(RETROSPOT TEA SET CERAMIC 11 PC ),0.000935,0.007016,0.000935,1.00,142.533333,0.000929,inf,0.993914
...,...,...,...,...,...,...,...,...,...,...
6634,(GARLAND WOODEN HAPPY EASTER),(LOVE BUILDING BLOCK WORD),0.000935,0.005613,0.000935,1.00,178.166667,0.000930,inf,0.995318
6635,(POLYESTER FILLER PAD 45x30cm),(RETROSPOT RED WASHING UP GLOVES),0.000935,0.003742,0.000935,1.00,267.250000,0.000932,inf,0.997191
6636,(RETROSPOT RED WASHING UP GLOVES),(POLYESTER FILLER PAD 45x30cm),0.003742,0.000935,0.000935,0.25,267.250000,0.000932,1.332086,1.000000
6637,(TRAVEL CARD WALLET VINTAGE ROSE ),(BUNDLE OF 3 RETRO NOTE BOOKS),0.000935,0.003742,0.000935,1.00,267.250000,0.000932,inf,0.997191


找出增益 (lift) 大於 1 的規則

In [7]:
rules[rules['lift'] > 1]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(FAMILY ALBUM WHITE PICTURE FRAME, 72 SWEETHEA...",(CHARLOTTE BAG SUKI DESIGN),0.000935,0.004210,0.000935,1.00,237.555556,0.000932,inf,0.996723
1,"(FAMILY ALBUM WHITE PICTURE FRAME, CHARLOTTE B...",(72 SWEETHEART FAIRY CAKE CASES),0.000935,0.004210,0.000935,1.00,237.555556,0.000932,inf,0.996723
2,"(72 SWEETHEART FAIRY CAKE CASES, CHARLOTTE BAG...",(FAMILY ALBUM WHITE PICTURE FRAME),0.000935,0.004210,0.000935,1.00,237.555556,0.000932,inf,0.996723
3,"(RETROSPOT TEA SET CERAMIC 11 PC , 72 SWEETHEA...",(NATURAL SLATE HEART CHALKBOARD ),0.000935,0.006080,0.000935,1.00,164.461538,0.000930,inf,0.994850
4,"(72 SWEETHEART FAIRY CAKE CASES, NATURAL SLATE...",(RETROSPOT TEA SET CERAMIC 11 PC ),0.000935,0.007016,0.000935,1.00,142.533333,0.000929,inf,0.993914
...,...,...,...,...,...,...,...,...,...,...
6634,(GARLAND WOODEN HAPPY EASTER),(LOVE BUILDING BLOCK WORD),0.000935,0.005613,0.000935,1.00,178.166667,0.000930,inf,0.995318
6635,(POLYESTER FILLER PAD 45x30cm),(RETROSPOT RED WASHING UP GLOVES),0.000935,0.003742,0.000935,1.00,267.250000,0.000932,inf,0.997191
6636,(RETROSPOT RED WASHING UP GLOVES),(POLYESTER FILLER PAD 45x30cm),0.003742,0.000935,0.000935,0.25,267.250000,0.000932,1.332086,1.000000
6637,(TRAVEL CARD WALLET VINTAGE ROSE ),(BUNDLE OF 3 RETRO NOTE BOOKS),0.000935,0.003742,0.000935,1.00,267.250000,0.000932,inf,0.997191
